# Grid Search

## Imports and Tensorboard

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..' + os.sep + '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
import numpy as np
import optuna
from experiment_parameters.TrainerFactory import dataset_model_dictionary
import xgboost as xgb

2025-11-07 13:26:26.070016: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-07 13:26:26.097655: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-07 13:26:26.491489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from experiment_parameters.model_builder.ModelBuilder import Director, get_training_configuration
from metrics.Metrics import DictOfMetrics
from metrics.Evaluator import evaluator

## Trial Factory

In [5]:
director = Director()

In [6]:
def get_parameters(trial, model_type):
    parameters = get_training_configuration(trial=trial, model_type=model_type)
    return parameters

In [7]:
def get_mlp(input_dim, num_classes, parameters):
    return director.create_mlp(input_parameters=input_dim, num_classes=num_classes, parameters=parameters)

In [8]:
def get_tabnet(input_dim, num_classes, parameters):
    return director.create_tabnet(input_parameters=input_dim, num_classes=num_classes, parameters=parameters).get_model()

In [9]:
def get_xgboost_tree(input_dim, num_classes, parameters):
    return director.create_xgboost(input_parameters=input_dim, num_classes=num_classes, parameters=parameters)

In [10]:
metric_list = ["CrossEntropyLoss"]

## MLP with Wine

In [ ]:
#from experiment_parameters.model_builder.Model import NeuralNetworkWine

dataset_factory = dataset_model_dictionary["wine"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()
#model = NeuralNetworkWine().get_model()

# tf_history = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), batch_size=1, epochs=20)

In [ ]:
def wine_mlp_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")
    
    # y_pred = ml_model.predict(X_test)
    # mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
    # mcc_metric.update_state(y_test, y_pred)
    # mcc_result = mcc_metric.result().numpy().tolist()
    
    # return loss, accuracy # , mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_wine", direction=['minimize'])
study.optimize(wine_mlp_optimization, n_trials=60)

## XGBoost Wine

In [ ]:
def wine_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
    dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500, verbose_eval=0) 
    eval_results = bst.eval(dmatrix_test)
    # print(f"Feature names: {bst.feature_names}")
    # print(f"Feature types: {bst.feature_types}")
    # log(INFO, f"Results: {eval_results}")
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
    
    # y_pred = bst.predict(X_test)
    # mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
    # mcc_metric.update_state(y_test, y_pred)
    # mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss#, accuracy#, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("xgboost_wine", direction=['minimize'])
study.optimize(wine_optimization, n_trials=60)

In [ ]:
from util.OptunaConnection import load_study

study = load_study("xgboost_wine")
trial_with_highest_accuracy = max(study.best_trials, key=lambda t: t.values[0])
parameters_dict = get_training_configuration(trial_with_highest_accuracy, "xgboost")
dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters_dict)
bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500)

xgb.plot_tree(bst)


In [ ]:
# from experiment_parameters.model_builder.TFModel import NeuralNetworkWine

dataset_factory = dataset_model_dictionary["wine"]
x_train, y_train = dataset_factory.get_dataset().get_training_data()
x_test, y_test = dataset_factory.get_dataset().get_test_data()
model = NeuralNetworkWine().get_model()

tf_history = model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=1, epochs=20)

## XGBoost with HAR

In [ ]:
dataset_factory = dataset_model_dictionary["har"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def har_xgboost_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
    dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500) 
    eval_results = bst.eval(dmatrix_test)
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
#     y_pred = ml_model.predict(X_test)
#     mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
#     mcc_metric.update_state(y_test, y_pred)
#     mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss #, accuracy, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("tabnet_har", direction=['minimize'])
study.optimize(har_xgboost_optimization, n_trials=60)

## Grid Search MLP HAR

In [ ]:
def har_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_har", direction=['minimize'])
study.optimize(har_optimization, n_trials=60)

## Grid Search XGBoost Adult

In [ ]:
from experiment_parameters.TrainerFactory import dataset_model_dictionary

dataset_factory = dataset_model_dictionary["adult"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def adult_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
    dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500) 
    eval_results = bst.eval(dmatrix_test)
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
#     y_pred = ml_model.predict(X_test)
#     mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
#     mcc_metric.update_state(y_test, y_pred)
#     mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss #, accuracy, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("xgboost_adult", direction=['minimize'])
study.optimize(adult_optimization, n_trials=60)

## Grid Search MLP Adult

In [ ]:
def adult_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_adult", direction=['minimize'])
study.optimize(adult_optimization, n_trials=60)

## Grid Search Adult XGBoost

## Grid Search CoverType XGBoost

In [ ]:
## Grid Search MLP Covertype
from experiment_parameters.TrainerFactory import dataset_model_dictionary

dataset_factory = dataset_model_dictionary["covertype"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def covertype_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
    dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500, early_stopping_rounds=10) 
    eval_results = bst.eval(dmatrix_test)
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
    
    return loss #, accuracy, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("xgboost_covertype", direction=['minimize'])
study.optimize(covertype_optimization, n_trials=60)

## Grid Search CoverType Dataset MLP

In [ ]:
def covertype_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_covertype", direction=['minimize'])
study.optimize(covertype_optimization, n_trials=60)

## Grid Search Heart TabNet

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary

dataset_factory = dataset_model_dictionary["heart"]
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def heart_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="tabnet")
    ml_model = get_tabnet(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("tabnet_heart", direction='minimize')
study.optimize(heart_optimization, n_trials=100)

## Grid Search Heart MLP

In [ ]:
def heart_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    loss, accuracy = ml_model.evaluate(X_test, y_test)
    
    y_pred = ml_model.predict(X_test)
    mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
    mcc_metric.update_state(y_test, y_pred)
    mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss #, accuracy, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study(name="mlp_heart", direction='minimize')
study.optimize(heart_optimization, n_trials=100)

## Grid Search Binary Heart Tabnet

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary

dataset_factory = dataset_model_dictionary["binary_heart"]
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def binary_heart_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="tabnet")
    ml_model = get_tabnet(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    loss, accuracy = ml_model.evaluate(X_test, y_test)
    
    y_pred = ml_model.predict(X_test)
    mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
    mcc_metric.update_state(y_test, y_pred)
    mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss , accuracy#, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("tabnet_binary_heart", direction=['minimize', 'maximize'])
study.optimize(binary_heart_optimization, n_trials=100)

## Grid Search Binary Heart MLP

In [ ]:
def binary_heart_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=20, batch_size=parameters["batch_size"]) 
    loss, accuracy = ml_model.evaluate(X_test, y_test)
    
    y_pred = ml_model.predict(X_test)
    mcc_metric = MatthewsCorrelationCoefficient(y_train.shape[1])
    mcc_metric.update_state(y_test, y_pred)
    mcc_result = mcc_metric.result().numpy().tolist()
    
    return loss, accuracy#, mcc_result

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_binary_heart", direction=['minimize', 'maximize'])
study.optimize(binary_heart_optimization, n_trials=100)

## New Adult

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary
from util.OptunaConnection import optuna_create_study

dataset_factory = dataset_model_dictionary["new_adult"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def new_adult_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [ ]:
from util.OptunaConnection import optuna_create_study

# 3. Create a study object and optimize the objective function.
study = optuna_create_study("mlp_new_adult", direction=['minimize'])
study.optimize(new_adult_optimization, n_trials=60)

In [ ]:
def new_adult_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=np.argmax(y_train, axis=1))
    dmatrix_test = xgb.DMatrix(X_test, label=np.argmax(y_test, axis=1))
    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=500, early_stopping_rounds=10) 
    eval_results = bst.eval(dmatrix_test)
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
    
    return loss #, accuracy, mcc_result

In [ ]:
study = optuna_create_study("xgboost_new_adult", direction=['minimize'])
study.optimize(new_adult_optimization, n_trials=60)

## Edge-IIOT-Coreset

In [11]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary
from util.OptunaConnection import optuna_create_study

dataset_factory = dataset_model_dictionary["edge-iot-coreset"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
display(X_train.shape)

In [12]:
def edge_iiot_coreset_optimization(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=y_train.shape[1], parameters=parameters)
    ml_model.fit(X_train, y_train, epochs=30, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("CrossEntropyLoss")

In [13]:
study = optuna_create_study("mlp_edge_iiot_coreset", direction=['minimize'])
study.optimize(edge_iiot_coreset_optimization, n_trials=30)

[I 2025-11-07 13:26:36,717] A new study created in RDB with name: mlp_edge_iiot_coreset
2025-11-07 13:26:36.830067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Epoch 1/30


2025-11-07 13:26:36.857490: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-07 13:26:36.858936: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-11-07 13:26:36.861253: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

295/589 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step - categorical_accuracy: 0.2326 - loss: 2.4840

I0000 00:00:1762518399.044981  452874 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

I0000 00:00:1762519784.056880  508157 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_621', 596 bytes spill stores, 596 bytes spill loads

I0000 00:00:1762519784.065796  508156 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_621', 592 bytes spill stores, 592 bytes spill loads

I0000 00:00:1762519784.175506  508155 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_661', 8 bytes spill stores, 8 bytes spill loads



537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.4970 - loss: 1.7734  

I0000 00:00:1762519785.531048  508245 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_46', 188 bytes spill stores, 188 bytes spill loads

I0000 00:00:1762519785.615540  508252 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_46', 212 bytes spill stores, 212 bytes spill loads



537/537 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - categorical_accuracy: 0.4974 - loss: 1.7723 - val_categorical_accuracy: 0.8544 - val_loss: 0.6711
Epoch 2/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step - categorical_accuracy: 0.8593 - loss: 0.6477 - val_categorical_accuracy: 0.8720 - val_loss: 0.5691
Epoch 3/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 585us/step - categorical_accuracy: 0.8715 - loss: 0.5753 - val_categorical_accuracy: 0.8756 - val_loss: 0.5400
Epoch 4/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 591us/step - categorical_accuracy: 0.8755 - loss: 0.5469 - val_categorical_accuracy: 0.8763 - val_loss: 0.5250
Epoch 5/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - categorical_accuracy: 0.8764 - loss: 0.5351 - val_categorical_accuracy: 0.8769 - val_loss: 0.5154
Epoch 6/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 575us/step - categorical_accuracy: 0.8776 - loss: 0.5212 - val_categorical_accuracy: 0.8775 - val_loss: 0.5079
Epoch 7/30
537/537 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - categorical_accuracy: 0.8787 - lo

/home/jose/miniconda3/envs/flower_py3-10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2981: UserWarning: The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.
  warnings.warn(
[I 2025-11-07 13:50:02,485] Trial 29 finished with value: 0.4528050302101597 and parameters: {'activation_l1': 'relu', 'batch_size': 498, 'decay_rate': 0.9, 'decay_steps': 2000, 'dropout_l1': 0.2, 'learning_rate_init': 0.0003838187805115565, 'n_units_l1': 117, 'num_layers': 1}. Best is trial 11 with value: 0.36115995079696805.


## Electric Consumption

In [ ]:
## Grid Search MLP HAR
from experiment_parameters.TrainerFactory import dataset_model_dictionary
from util.OptunaConnection import optuna_create_study

metric_list = ["MAE"]

dataset_factory = dataset_model_dictionary["electric-consumption"]()
X_train, y_train = dataset_factory.get_dataset().get_training_data()
X_test, y_test = dataset_factory.get_dataset().get_test_data()

In [ ]:
def electric_consumption_optimisation(trial):
    parameters = get_parameters(trial=trial, model_type="mlp")
    shape: int
    try: shape = y_train.shape[1] 
    except: shape = 1
    ml_model = get_mlp(input_dim=X_train.shape[1], num_classes=shape, parameters=parameters)
    ml_model.fit(X_train, y_train, batch_size=parameters["batch_size"]) 
    metrics: DictOfMetrics = evaluator(X_train, y_train, ml_model, metric_list)
    return metrics.get_value_of_metric("MAE")

In [ ]:
study = optuna_create_study("mlp_electric_consumption", direction=['minimize'])
study.optimize(electric_consumption_optimisation, n_trials=30)

In [ ]:
def electric_consumption_optimisation_xgboost(trial):
    parameters = get_parameters(trial=trial, model_type="xgboost")
    dmatrix = xgb.DMatrix(X_train, label=y_train)
    dmatrix_test = xgb.DMatrix(X_test, label=y_test)
    shape: int
    try: shape = y_train.shape[1] 
    except: shape = 1

    xgboost_hyperparameters = get_xgboost_tree(input_dim=X_train.shape[1], num_classes=shape, parameters=parameters)
    bst = xgb.train(xgboost_hyperparameters, dmatrix, evals=[(dmatrix_test, "validate"), (dmatrix, "train")], num_boost_round=2000, early_stopping_rounds=5) 
    eval_results = bst.eval(dmatrix_test)
    loss = round(float(eval_results.split("\t")[1].split(":")[1]), 4)
    return loss

In [ ]:
study = optuna_create_study("xgboost_electric_consumption", direction=['minimize'])
study.optimize(electric_consumption_optimisation_xgboost, n_trials=30)